In [1]:
from google.genai import types
from google.adk.agents import Agent
from google.adk.events import Event
from google.adk.runners import Runner
from google.adk.models.google_llm import Gemini
from google.adk.tools.tool_context import ToolContext
from google.adk.sessions import InMemorySessionService, Session
from google.adk.apps.app import App, ResumabilityConfig

In [2]:
retry_config=types.HttpRetryOptions(attempts=3, exp_base=5, initial_delay=1, http_status_codes=[429, 500, 503, 504])
model = Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config)

In [3]:
def approve_idea(tool_context: ToolContext, idea: str) -> dict:
    """Waits for user to approve the idea.

    Args:
        idea [str]: summary of the idea to approve

    Returns:
        Dictionary with approval status: "approved", "rejected", or "pending".
    """
    # On first time run, wait for human approval - pause here.
    if not tool_context.tool_confirmation:
        tool_context.request_confirmation(hint=f"Idea: {idea}. Do you want to approve?", payload={"idea": idea})
        return {"status": "pending", "message": f"Idea requires approval"}
    # On the resumption, handle the approval response.
    if tool_context.tool_confirmation.confirmed:
        return {"status": "approved", "message": f"Idea approved: {idea}"}
    return {"status": "rejected", "message": f"Idea rejected: {idea}"}

In [4]:
idea_agent = Agent(
    name="IdeaAgent",
    model=model,
    instruction="""
    You are a creative idea generator for short videos for social media.
    1. Create unique and innovative idea based on user prompts.
    2. Use the approve_idea tool to get human approval for each idea.
    3. If the order status is 'pending', inform the user that approval is required.
    4. After receiving the final result, provide a clear summary of idea.
    """,
    tools=[approve_idea],
)

In [5]:
idea_app = App(
    name="IdeaApp",
    root_agent=idea_agent,
    resumability_config=ResumabilityConfig(is_resumable=True),
)

/tmp/ipykernel_111301/4069985334.py:4: UserWarning: [EXPERIMENTAL] ResumabilityConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  resumability_config=ResumabilityConfig(is_resumable=True),


In [6]:
session_service = InMemorySessionService()

In [7]:
runner = Runner(app=idea_app, session_service=session_service)

In [ ]:
async def run(query: str):
    user_id = "user_id"
    session: Session = await runner.session_service.create_session(app_name=idea_app.name, user_id=user_id)

    print(f"User > {query}")

    # Send initial request to the agent
    events: list[Event] = [event async for event in runner.run_async(
        user_id=user_id,
        session_id=session.id,
        new_message=types.Content(role="user", parts=[types.Part(text=query)]))]

    # Find `adk_request_confirmation` event returned by agent
    approval_id, invocation_id = None, None
    for event in events:
        if not event.content: continue
        if not event.content.parts: continue
        for part in event.content.parts:
            if part.function_call and part.function_call.name == "adk_request_confirmation":
                approval_id = part.function_call.id
                invocation_id = event.invocation_id
                break
    assert approval_id is not None and invocation_id is not None, "No approval request found"

    # Handle approval 
    approve = input("HumanInTheLoop > Do you approve the idea? (y/n): ").strip().lower()
    print(f"HumanInTheLoop > Decision: {'✅' if approve == 'y' else '❌'}")

    # Resume the agent with the approval decision
    decision = types.FunctionResponse(
        id=approval_id,
        name="adk_request_confirmation",
        response={"confirmed": approve == 'y'})
    events: list[Event] = [event async for event in runner.run_async(
        user_id=user_id,
        session_id=session.id,
        new_message=types.Content(role="user", parts=[types.Part(function_response=decision)]))]
    for event in events:
        if not event.content: continue
        if not event.content.parts: continue
        for part in event.content.parts:
            if not part.text: continue
            print(f"Agent > {part.text}")

In [11]:
await run("Create a short video about health risks")

User > Create a short video about health risks
HumanInTheLoop > Do you approve the idea? (y/n):
HumanInTheLoop > Decision: ✅
Agent > I'm glad you liked the idea! Here's a summary of the short video concept:

The video will visually represent the health risks of prolonged sitting. It starts with a person working at a desk, and as the day progresses, they begin to physically merge with their chair. This transformation can be shown through subtle to more dramatic effects, like their legs becoming chair legs or their torso blending into the seat. The video aims to be a creative and slightly humorous, yet impactful, way to highlight the dangers of a sedentary lifestyle.


In [12]:
await run("Create a short video about health risks")

User > Create a short video about health risks
HumanInTheLoop > Do you approve the idea? (y/n):
HumanInTheLoop > Decision: ❌
Agent > I'm sorry, that idea has been rejected. Can you please provide another prompt for a short video about health risks?
